# Dashboard prototype
This is making a dashboard to visualise data using Dash and Plotly

In [1]:
!pip3 install jupyter-dash

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 101 kB 16.1 MB/s ta 0:00:01
     |████████████████████████████████| 10.2 MB 63.8 MB/s eta 0:00:01
     |████████████████████████████████| 15.2 MB 74.3 MB/s eta 0:00:01


In [6]:
import json
#from dash import Dash, html, dcc       Dash to be tried later as a standalone app
#from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

In [21]:
df = pd.read_csv('../data/interim/CMS_2019_to_2022_ENGLISH.csv', index_col=0)

df.head()

/home/wojciech/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning:

Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.



,id,name,shortTitle,text,teaser,shortTeaser,kicker,regions,keywords,keywordStrings,...,articles,isOpinion,geographicLocations,contentAssociations,mainContentImageLink,images,externalLinks,topStory,language,ts_lastModifiedDate
60278,46912921,"NASA spacecraft enters asteroid Bennu's orbit,...","NASA craft enters asteroid orbit, sets records",<p>NASA's unmanned spacecraft OSIRIS-REx enter...,The OSIRIS-REx spacecraft had arrived at the l...,NASA said its OSIRIS-REx spacecraft succesfull...,NaN,[],"[{'name': 'NASA'}, {'name': 'OSIRIS-REx'}, {'n...","['NASA', 'OSIRIS-REx', 'Bennu', 'asteroid']",...,"[{'name': 'After two-year journey, NASA Osiris...",False,[],"[{'id': 47376129}, {'id': 47376131}, {'id': 47...",{'description': None},[{'name': 'Sonde «Osiris Rex» nach zwei Jahren...,[{'name': 'DW Newsletter'}],False,NaN,2019-01-01 03:57:28.904000+00:00
60279,46911356,UK brings in boats to patrol English Channel a...,UK deploys boats amid Channel migration spike,<p>Britain will recall two overseas border pat...,The UK is withdrawing patrol ships from overse...,The UK is withdrawing patrol ships from overse...,NaN,[],"[{'name': 'English Channel'}, {'name': 'migrat...","['English Channel', 'migration', 'boats', 'ill...",...,[{'name': '40 migrants rescued from boats in E...,False,[],"[{'id': 47375206}, {'id': 47375208}, {'id': 47...",{'description': None},[{'name': 'Großbritanien | Zahl der Bootsflüch...,[],False,NaN,2019-01-01 06:11:50.527000+00:00
60280,46909694,"In Brazil, Jair Bolsonaro tries to unite the e...",Can Bolsonaro unite Brazil's motley right?,<p>Politics without ideologies and party cliqu...,Brazil is inaugurating President Jair Bolsonar...,"In his campaign, Jair Bolsonaro targeted appea...",NaN,[],"[{'name': 'Brazil'}, {'name': 'Jair Bolsonaro'...","['Brazil', 'Jair Bolsonaro', 'Chicago economic...",...,"[{'name': ""Bolsonaro's policies divide Afro-Br...",False,[],"[{'id': 47373913}, {'id': 47374152}, {'id': 47...",{'description': None},[{'name': 'Brasilien - Präsident Jair Bolsonar...,[{'name': ''}],False,NaN,2019-01-01 06:14:35.563000+00:00
60281,46912694,Tokyo: Man deliberately drives van into NYE cr...,Man drives car into Tokyo NYE crowd,<p>At least nine people were injured when a ma...,"A man with an ""intent to murder"" has driven a ...","A man with an ""intent to murder"" has driven a ...",NaN,[],"[{'name': 'Japan'}, {'name': 'Tokyo'}, {'name'...","['Japan', 'Tokyo', 'Harajuku', 'attack']",...,[],False,[],"[{'id': 47376009}, {'id': 47376011}, {'id': 47...",{'description': None},[{'name': 'Japan | Autoangriff in Tokyo'}],[],False,NaN,2019-01-01 08:26:11.599000+00:00
60282,46910092,Bangladesh opposition leader: International co...,International community 'shouldn't recognize n...,"<div class=""vjs-wrapper embed big ""><h2 aria-l...","In an exclusive interview with DW, Kamal Hossa...",Bangladeshi opposition leader Kamal Hossain sl...,NaN,[],"[{'name': 'Asia'}, {'name': 'Bangladesh'}, {'n...","['Asia', 'Bangladesh', 'elections', 'Kamal Hos...",...,"[{'name': ""Opinion: Bangladesh's missed chance...",False,[],"[{'id': 47374183}, {'id': 47374251}, {'id': 47...",{'description': None},"[{'name': 'Dr. Kamal Hossain '}, {'name': 'Pre...",[],False,NaN,2019-01-01 09:05:00.736000+00:00


In [25]:
keys_times = df[['ts_lastModifiedDate', 'keywordStrings']]
keys_times['keywordStrings'] = keys_times['keywordStrings'].apply(literal_eval)        #This ensures keywordStrings is an actual list, not just 'look-alike'
keys_times = keys_times.explode('keywordStrings')
keys_times.head()

/home/wojciech/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ts_lastModifiedDate,keywordStrings
60278,2019-01-01 03:57:28.904000+00:00,NASA
60278,2019-01-01 03:57:28.904000+00:00,OSIRIS-REx
60278,2019-01-01 03:57:28.904000+00:00,Bennu
60278,2019-01-01 03:57:28.904000+00:00,asteroid
60279,2019-01-01 06:11:50.527000+00:00,English Channel


In [41]:
top10 = keys_times['keywordStrings'].value_counts().nlargest(10)
print(top10.index)

Index(['Germany', 'coronavirus', 'COVID-19', 'China', 'Russia', 'Donald Trump',
       'Asia', 'US', 'EU', 'Bundesliga'],
      dtype='object')


In [69]:
keys_times['date']=pd.to_datetime(keys_times['ts_lastModifiedDate'])

keys_times["month"] = keys_times["date"].dt.month
keys_times["year"] = keys_times["date"].dt.year

grouped_keys_times = keys_times.groupby([keys_times.keywordStrings, keys_times.month, keys_times.year]).count().reset_index()
grouped_keys_times = grouped_keys_times.rename(columns={"date": "count"})
grouped_keys_times['yearmonth'] = pd.to_datetime(dict(year=grouped_keys_times.year, month=grouped_keys_times.month, day='01'))
grouped_keys_times.head()

,keywordStrings,month,year,ts_lastModifiedDate,count,yearmonth
0,Refugees,2,2021,1,1,2021-02-01
1,Refugees,3,2021,2,2,2021-03-01
2,Refugees,5,2021,1,1,2021-05-01
3,Refugees,8,2021,1,1,2021-08-01
4,Refugees,9,2021,1,1,2021-09-01


In [71]:
fig_df = grouped_keys_times.loc[grouped_keys_times['keywordStrings'].isin(top10.index)]
fig_df = fig_df.sort_values(by="yearmonth")

fig = px.line(fig_df, x='yearmonth', y="count", color="keywordStrings")
fig.show()